In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_cities = pd.read_csv('/content/drive/MyDrive/NOVARTIS NEST - BioLogits/Data/Population_extracted.csv')
df_actual = pd.read_excel('/content/drive/MyDrive/NOVARTIS NEST - BioLogits/Data/usecase_4_.xlsx')
df_cities.shape, df_actual.shape

((20676, 7), (20676, 29))

In [ ]:
df_cities.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'NCT Number', 'Locations', 'Cities',
       'no_of_sites', 'population'],
      dtype='object')

In [ ]:
df = pd.merge(df_cities, df_actual, on='NCT Number', how='inner')
df.shape

(20676, 35)

In [ ]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'NCT Number', 'Locations_x', 'Cities',
       'no_of_sites', 'population', 'Study Title', 'Study URL', 'Study Status',
       'Brief Summary', 'Study Results', 'Conditions', 'Interventions',
       'Primary Outcome Measures', 'Secondary Outcome Measures',
       'Other Outcome Measures', 'Sponsor', 'Collaborators', 'Sex', 'Age',
       'Phases', 'Enrollment', 'Funder Type', 'Study Type', 'Study Design',
       'Other IDs', 'Start Date', 'Primary Completion Date', 'Completion Date',
       'First Posted', 'Results First Posted', 'Last Update Posted',
       'Locations_y', 'Study Recruitment Rate'],
      dtype='object')

In [ ]:
df = df.rename(columns={'Unnamed: 0': 'index'})

In [ ]:
exploded = df.explode('Cities')

def get_overlapping_indices(group):
    starts = group['Start Date'].values
    ends = group['Completion Date'].values

    overlap_matrix = (starts[:, None] <= ends) & (starts <= ends[:, None]).T
    np.fill_diagonal(overlap_matrix, False)

    return pd.Series(
        [set(group['index'][overlap_matrix[i]].values) for i in range(len(group))],
        index=group.index
    )

exploded['overlaps'] = exploded.groupby('Cities', group_keys=False).apply(get_overlapping_indices)


df = df.merge(
    exploded.groupby('index')['overlaps'].agg(lambda x: set().union(*x)),
    left_on='index',
    right_index=True
)
df['competition'] = df['overlaps'].apply(len)
df = df.drop(columns=['index', 'overlaps'])

print(df[['Cities', 'Start Date', 'Completion Date', 'competition']])

<ipython-input-37-3f27db04eaf0>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  exploded['overlaps'] = exploded.groupby('Cities', group_keys=False).apply(get_overlapping_indices)


                                                  Cities Start Date  \
0                                         ['Pittsburgh'] 2014-05-12   
1      ['Providence', 'Mumbai', 'Pune', 'Chennai', 'K... 2014-02-20   
2      ['Gent', 'Brno', 'Praha 5 - Motol', 'København... 2014-03-03   
3                                   ['Fès', 'Marrakech'] 2018-07-21   
4      ['Bandarban', 'Manaus', 'Porto Velho', 'Oddar ... 2014-04-24   
...                                                  ...        ...   
20671                                      ['Groningen'] 2021-05-17   
20672  ['Duarte', 'Stanford', 'Aurora', 'Washington',... 2015-08-04   
20673  ['Busan', 'Seoul', 'Seoul', 'Seoul', 'Seoul', ... 2018-08-30   
20674  ['Birmingham', 'Fayetteville', 'New Haven', 'N... 2014-01-14   
20675  ['Berlin', 'Berlin', 'Berlin', 'Berlin', 'Boch... 2018-03-07   

      Completion Date  competition  
0          2015-06-02           19  
1          2015-10-17            0  
2          2017-05-09            0  

In [ ]:
df_check = df[df['Cities']=="['Pittsburgh']"]
df_check = df_check[['Cities', 'competition', 'Start Date', 'Completion Date']]
df_check

,Cities,competition,Start Date,Completion Date
0,['Pittsburgh'],19,2014-05-12,2015-06-02
417,['Pittsburgh'],19,2014-07-24,2020-07-17
455,['Pittsburgh'],19,2014-10-28,2016-10-31
2477,['Pittsburgh'],17,2018-01-31,2019-05-28
3111,['Pittsburgh'],18,2016-07-13,2021-04-23
3595,['Pittsburgh'],18,2016-03-21,2018-06-21
5680,['Pittsburgh'],16,2018-11-15,2020-02-25
6116,['Pittsburgh'],17,2017-12-01,2024-03-01
6237,['Pittsburgh'],17,2017-12-13,2019-04-30
6609,['Pittsburgh'],17,2018-03-14,2023-06-05


In [ ]:
df = df[['NCT Number','competition']]
df

,NCT Number,competition
0,NCT00900809,19
1,NCT01113515,0
2,NCT01288573,0
3,NCT01336660,0
4,NCT01376167,0
...,...,...
20671,NCT04883333,57
20672,NCT02453087,0
20673,NCT03850587,0
20674,NCT01981187,0


In [ ]:
df.to_csv('/content/drive/MyDrive/NOVARTIS NEST - BioLogits/Data/competition.csv', index=False)

In [ ]:
## time

df_actual['Start Date'] = pd.to_datetime(df_actual['Start Date'])
df_actual['Completion Date'] = pd.to_datetime(df_actual['Completion Date'])

# Calculate duration in days
df_actual['duration'] = (df_actual['Completion Date'] - df_actual['Start Date']).dt.days

In [ ]:
df_actual[['Start Date', 'Completion Date', 'duration']]

,Start Date,Completion Date,duration
0,2014-05-12,2015-06-02,386.0
1,2014-02-20,2015-10-17,604.0
2,2014-03-03,2017-05-09,1163.0
3,2018-07-21,2018-11-15,117.0
4,2014-04-24,2016-11-18,939.0
...,...,...,...
20671,2021-05-17,2022-07-18,427.0
20672,2015-08-04,2019-07-12,1438.0
20673,2018-08-30,2020-09-07,739.0
20674,2014-01-14,2015-10-13,637.0


In [ ]:
df_actual = df_actual[['NCT Number', 'duration']]
df_actual

,NCT Number,duration
0,NCT00900809,386.0
1,NCT01113515,604.0
2,NCT01288573,1163.0
3,NCT01336660,117.0
4,NCT01376167,939.0
...,...,...
20671,NCT04883333,427.0
20672,NCT02453087,1438.0
20673,NCT03850587,739.0
20674,NCT01981187,637.0


In [ ]:
df_actual.to_csv('/content/drive/MyDrive/NOVARTIS NEST - BioLogits/Data/duration.csv', index=False)